# Single Shot Detection

## Setup Collab

### Install nessary packages

#### torchvision install

In [ ]:
! pip install torch torchvision

#### albumentations install

In [ ]:
! pip install -U albumentations

#### opencv-python-headless install

In [ ]:
! pip install opencv-python-headless